In [1]:
%load_ext autoreload
%autoreload 2

import sys
import json
import torch
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, mean_absolute_error, r2_score
from scipy.stats import spearmanr, pearsonr

sys.path.append('..')
sys.path.append('pLMtrainer')
from pLMtrainer import utils

#torch.set_float32_matmul_precision('medium')

parquet_path = "../data/frustration/v8_frustration_v2.parquet.gzip"

/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pq.read_table(parquet_path).to_pandas()
df.head(2)

,proteinID,full_seq,res_seq,res_idx,frst_idx,frst_class,frst_class_3,split_test,cath_id,cath_T_id,split_old,split_0,split_1,split_2,split_3
0,1abrA01,EDRPIKFSTEGATSQSYKQFIEALRERLRGGLIHDIPVLPDPTTLQ...,"[E, D, R, P, I, K, F, S, T, E, G, A, T, S, Q, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1.25, -0.466, 0.868, -2.941, 1.638, -0.771, 1...","[f8, f12, f9, f17, f7, f12, f7, f12, f11, f12,...","[2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 1, 2, 2, 0, 0, ...",train,3.40.420.10,3.40.420,None,train,train,val,train
1,1apaA01,APTLEINTITFDVGNATINKYATFMKSIHNQAKDPTLKCYGIPMLP...,"[I, N, T, I, T, F, D, V, G, N, A, T, I, N, K, ...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[1.942, -0.924, 0.115, 1.605, 0.26, 1.182, -0....","[f6, f13, f10, f7, f10, f8, f12, f8, f12, f8, ...","[2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 0, ...",train,3.40.420.10,3.40.420,None,train,train,val,train


In [3]:
train_df = df[df["split_test"] == "train"]
val_df = df[df["split_test"] == "val"]
test_df = df[df["split_test"] == "test"]

In [ ]:
# sum of seqs in each set
len(df),len(train_df), len(val_df), len(test_df)

(983212, 886837, 48610, 47765)

In [ ]:
# proportion of seqs in each set
len(train_df) / len(df), len(val_df) / len(df), len(test_df) / len(df)

(0.9019794306822944, 0.04943999869814445, 0.04858057061956119)

In [ ]:
#total num frst res
np.concatenate(df["frst_idx"].values).shape

(186477919,)

In [8]:
# amount of topologies in each set
df["cath_id"].nunique(), train_df["cath_id"].nunique(), val_df["cath_id"].nunique(), test_df["cath_id"].nunique()

(1200, 1083, 61, 56)

In [9]:
# proportions of frst classes
np.bincount(np.concatenate(df['frst_class_3'].values)) / np.sum(np.bincount(np.concatenate(df['frst_class_3'].values)))

array([0.12543113, 0.48395942, 0.39060945])

In [10]:
# proportions of frst classes
np.bincount(np.concatenate(train_df['frst_class_3'].values)) / np.sum(np.bincount(np.concatenate(train_df['frst_class_3'].values))), \
np.bincount(np.concatenate(val_df['frst_class_3'].values)) / np.sum(np.bincount(np.concatenate(val_df['frst_class_3'].values))), \
np.bincount(np.concatenate(test_df['frst_class_3'].values)) / np.sum(np.bincount(np.concatenate(test_df['frst_class_3'].values)))

(array([0.12527583, 0.48368734, 0.39103683]),
 array([0.12797107, 0.48267788, 0.38935105]),
 array([0.12579304, 0.48927343, 0.38493352]))

In [ ]:
# amount of residues that are in the first 512 residues of a protein sequence. 
(np.concatenate(df["res_idx"]) <= 512).sum()

np.int64(172125066)

In [14]:
#and its proportion
(np.concatenate(df["res_idx"]) <= 512).sum() / np.concatenate(df["res_idx"]).shape[0]

np.float64(0.9230318899043484)